In [1]:
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import RandomizedSearchCV
from time import time

In [2]:
#read table: train as total training dataset, train_re as resampled training
#dataset for modeling,test as test dataset

train=pd.read_table('training.txt')  #80% of original data randomly selected on SAS
test=pd.read_table('testing.txt')  #Left 20%
#Unlabeled data was resampled on SAS to be 
#Count of Resampled Unlabeled + Count of Labeled Negative = Count of Labeled Positive * 4
train_re=pd.read_table('training_re.txt') 

In [3]:
#extract response column
target = train_re.iloc[:,[293]]

#extract train matrix
tr = train_re.iloc[:,1:293]

#extract test matrix
tt = test.iloc[:,2:294]

#extract response variable for model, strange data type for python 
target=target['PERT_Y']

In [4]:
#set random forest tree as 500
rf=RandomForestClassifier(n_estimators=500)
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [5]:
#hyper parameter options, could also apply to other model. 
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 20),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

In [7]:
#search 50 times  
seed = np.random.seed(1234)
n_iter_search = 50
random_search = RandomizedSearchCV(rf, param_distributions=param_dist,
                                   n_iter=n_iter_search, n_jobs=-1, verbose=100)

start = time()
random_search.fit(tr,target )
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

report(random_search.cv_results_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Pickling array (shape=(292,), dtype=object).
Memmaping (shape=(292, 8350), dtype=int64) to new file C:\Users\motoharu.dei\AppData\Local\Temp\joblib_memmaping_pool_6884_190537176\6884-107636216-4feb91f2aaa8da14794982997381cecb.pkl
Pickling array (shape=(292,), dtype=object).
Pickling array (shape=(8350,), dtype=object).
Pickling array (shape=(5566,), dtype=int32).
Pickling array (shape=(2784,), dtype=int32).
Pickling array (shape=(292,), dtype=object).
Memmaping (shape=(292, 8350), dtype=int64) to old file C:\Users\motoharu.dei\AppData\Local\Temp\joblib_memmaping_pool_6884_190537176\6884-107636216-4feb91f2aaa8da14794982997381cecb.pkl
Pickling array (shape=(292,), dtype=object).
Pickling array (shape=(8350,), dtype=object).
Pickling array (shape=(5566,), dtype=int32).
Pickling array (shape=(2784,), dtype=int32).
Pickling array (shape=(292,), dtype=object).
Memmaping (shape=(292, 8350), dtype=int64) to old file C:\Users\motohar

In [8]:
#RandomizedSearchCV took 305.50 seconds for 50 candidates parameter settings.
#Model with rank: 1
#Mean validation score: 0.902 (std: 0.014)
#Parameters: {'max_depth': None, 'min_samples_leaf': 5, 'max_features': 18, 
#'criterion': 'gini', 'bootstrap': False, 'min_samples_split': 9}

#build random forest model after random search.
rf=RandomForestClassifier(criterion='gini', max_depth=None, min_samples_split=9, 
                       min_samples_leaf=5,max_features=18,bootstrap=False,random_state=1234)

#fit the model
rf.fit(tr, target)

#apply to original train data from resample train data model result
tr1 = train.iloc[:,1:293]
predicted_probs1 = rf.predict_proba(tr1)
a= predicted_probs1[:,1]
np.mean(a)
b= predicted_probs1[:,0]
np.mean(b)
a1=sorted(a, reverse=True)
a2=pd.DataFrame(a1)

#get cut-off probability at labeling 4-fold positive to original positive (1670*4=6680)
cr=a2.iloc[target.value_counts()["Y"]*4-1,:]
print(cr)

0    0.620635
Name: 6679, dtype: float64


In [9]:
#Synchronize with Jeff's Scoring Functions
from sklearn.metrics import accuracy_score, precision_score, recall_score, \
    f1_score, roc_auc_score, average_precision_score, brier_score_loss, \
    fbeta_score, confusion_matrix
from creonmetrics import labeled_metric, assumed_metric, pu_score, \
    pr_one_unlabeled, brier_score_partial_loss

y_true = np.transpose(test.iloc[:,0].values)
y_prob = rf.predict_proba(tt)

y_pred=np.array([1 if x >= 0.620635 else 0 for x in y_prob[:,1]])

In [10]:
import collections
data = {'labeled_acc' : labeled_metric(y_true, y_pred, accuracy_score),
        'labeled_prec' : labeled_metric(y_true, y_pred, precision_score),
        'labeled_recall' : labeled_metric(y_true, y_pred, recall_score),
        'labeled_f1' : labeled_metric(y_true, y_pred, f1_score),
        'labeled_roc_auc' : labeled_metric(y_true, y_pred, roc_auc_score),
        'labeled_avg_prec' : labeled_metric(y_true, y_pred, average_precision_score),
        'labeled_brier' : labeled_metric(y_true, y_prob, brier_score_loss),
        'labeled_brier_pos' : labeled_metric(y_true, y_prob, brier_score_partial_loss, label=1),
        'labeled_brier_neg' : labeled_metric(y_true, y_prob, brier_score_partial_loss, label=0),
        'confusion_matrix_lab' : labeled_metric(y_true, y_pred, confusion_matrix),
        'pr_one_unlabeled' : pr_one_unlabeled(y_true, y_pred),
        'assumed_brier' : assumed_metric(y_true, y_prob, brier_score_loss),
        'assumed_brier_neg' : assumed_metric(y_true, y_prob, brier_score_partial_loss, label=0),
        'assumed_f1' : assumed_metric(y_true, y_pred, f1_score),
        'assumed_f1beta10' : assumed_metric(y_true, y_pred, fbeta_score, beta=10),
        'confusion_matrix_un' : assumed_metric(y_true, y_pred, confusion_matrix),
        'pu_score' : pu_score(y_true, y_pred),
        }
data_s = collections.OrderedDict(sorted(data.items()))
for k, v in data_s.items():
    print(k, ': ', v)


assumed_brier :  0.0331460952462
assumed_brier_neg :  0.0320835681346
assumed_f1 :  0.187751004016
assumed_f1beta10 :  0.435445197584
confusion_matrix_lab :  [[210   6]
 [231 187]]
confusion_matrix_un :  [[86335  1387]
 [  231   187]]
labeled_acc :  0.6261829653
labeled_avg_prec :  0.890316825227
labeled_brier :  0.184715372179
labeled_brier_neg :  0.0465160873385
labeled_brier_pos :  0.25612935669
labeled_f1 :  0.612111292962
labeled_prec :  0.968911917098
labeled_recall :  0.447368421053
labeled_roc_auc :  0.709795321637
pr_one_unlabeled :  0.015781774964
pu_score :  11.2072476215


In [11]:
rfimp=pd.DataFrame(rf.feature_importances_, index=tt.columns).sort_values(by=0, ascending=False)
rfimp.head(50)

,0
DIAG_FLAG4_Sum,0.257138
DIAG_FLAG5_Sum,0.144012
DIAG_FLAG75_Sum,0.036151
ndc_cat87_Sum,0.035965
ndc_cat58_Sum,0.032024
CPT_FLAG9_Sum,0.023963
age,0.016386
CPT_FLAG44_Sum,0.013212
DIAG_FLAG38_Sum,0.013183
CPT_FLAG43_Sum,0.011318
